# 📊 데이터 통합 프로젝트

Welcome과 Quickpoll 데이터를 통합하여 통합 데이터셋을 생성하는 프로세스

## 🎯 프로젝트 개요

| 항목 | 내용 |
|------|------|
| **목적** | Welcome 파일 2개와 Quickpoll 파일 35개를 mb_sn을 기준으로 통합 |
| **데이터 소스** | ../data/raw/ 폴더의 Excel 파일들 |
| **통합 기준** | mb_sn 컬럼을 사용한 Outer Join |
| **결과** | 36,113명의 패널 데이터 통합 |

## 📋 데이터 처리 특징

### 🔹 Welcome 파일
- **헤더 위치**: 첫 번째 행 (header=0)
- **파일 수**: 2개
- **내용**: 기본 패널 정보 (인구통계, 기본 설문)

### 🔹 Quickpoll 파일
- **헤더 위치**: 두 번째 행 (header=1)
- **파일 수**: 35개
- **내용**: 설문 응답 데이터 (날짜별 설문)

### 🔹 통합 과정
- '고유번호' 컬럼을 'mb_sn'으로 변경
- Outer Join으로 모든 데이터 보존
- 중복 컬럼 자동 정리

## 📁 노트북 구조

1. **라이브러리 설정** - 필요한 패키지 import
2. **파일 경로 설정** - 데이터 파일 경로 및 목록 확인
3. **Welcome 데이터 사전 생성** - Welcome 해설지 처리
4. **Welcome 파일 통합** - 기본 패널 데이터 통합
5. **Quickpoll 해설지 추출** - Quickpoll 해독표 생성
6. **Quickpoll 파일 통합** - 설문 응답 데이터 통합
7. **중복 데이터 처리** - 데이터 품질 개선
8. **최종 결과 저장** - 통합된 데이터셋 저장


## 📚 1. 라이브러리 설정

데이터 처리에 필요한 라이브러리들을 import합니다.


In [39]:
import pandas as pd
import glob  # 파일 경로를 쉽게 다룰 수 있게 해주는 라이브러리
import os    # 운영체제와 상호작용하는 라이브러리
import gc    # 가비지 컬렉션을 위한 라이브러리


## 📁 2. 파일 경로 설정

데이터 파일들이 위치한 경로를 설정하고 파일 목록을 확인합니다.


In [40]:
# 경로 설정 및 파일 목록 불러오기
base_path = '../data/raw/'
welcome_path = os.path.join(base_path, 'Welcome/*.xlsx')
quickpoll_path = os.path.join(base_path, 'Quickpoll/*.xlsx')

welcome_files = glob.glob(welcome_path)
quickpoll_files = glob.glob(quickpoll_path)

print(f"Welcome 파일 경로: {welcome_path}")
print(f"Quickpoll 파일 경로: {quickpoll_path}")
print(f"Welcome 파일 개수: {len(welcome_files)}개")
print(f"Quickpoll 파일 개수: {len(quickpoll_files)}개")


Welcome 파일 경로: ../data/raw/Welcome/*.xlsx
Quickpoll 파일 경로: ../data/raw/Quickpoll/*.xlsx
Welcome 파일 개수: 2개
Quickpoll 파일 개수: 35개


## 📋 3. Welcome 데이터 사전 생성

Welcome 파일들의 해설지를 처리하여 데이터 사전을 생성합니다.


In [41]:
# Welcome 1의 두 번째 시트(해설지) 처리
print("=== Welcome 1 해설지 처리 ===")
try:
    df_label1_raw = pd.read_excel(welcome_files[0], sheet_name=1)
    print(f"Welcome 1 해설지 원본 크기: {df_label1_raw.shape}")
    print(f"컬럼명: {df_label1_raw.columns.tolist()}")
    
    # 첫 번째 컬럼을 '코드', 두 번째 컬럼을 '내용'으로 설정
    df_label1_raw.columns = ['코드', '내용'] + list(df_label1_raw.columns[2:])
    
    processed_rows = []
    current_question_code = None
    current_question_text = None
    
    for index, row in df_label1_raw.iterrows():
        code_val = str(row['코드']).strip()
        content_val = str(row['내용']).strip()
        
        # 질문 코드인지 확인 (숫자가 아닌 경우)
        if code_val and not code_val.isdigit() and code_val != 'nan':
            current_question_code = code_val
            current_question_text = content_val
        # 보기 번호인지 확인 (숫자인 경우)
        elif code_val and code_val.isdigit() and current_question_code:
            processed_rows.append({
                '질문코드': 'W1_' + current_question_code,
                '질문내용': current_question_text,
                '보기번호': int(code_val),
                '보기내용': content_val
            })
    
    df_label1 = pd.DataFrame(processed_rows)
    print(f"✅ Welcome 1 해설지 처리 완료: {len(df_label1)}개 항목")
    
except Exception as e:
    print(f"❌ Welcome 1 해설지 처리 중 오류: {e}")
    df_label1 = pd.DataFrame()

# Welcome 2의 두 번째 시트(해설지) 처리
print("\n=== Welcome 2 해설지 처리 ===")
try:
    df_label2 = pd.read_excel(welcome_files[1], sheet_name=1)
    print(f"Welcome 2 해설지 원본 크기: {df_label2.shape}")
    print(f"컬럼명: {df_label2.columns.tolist()}")
    
    # 컬럼명 매핑
    column_mapping = {
        '변수명': '질문코드',
        '내용': '질문내용',
        '값': '보기번호', 
        '값설명': '보기내용'
    }
    
    # 실제 컬럼명 확인 후 매핑
    for old_col, new_col in column_mapping.items():
        if old_col in df_label2.columns:
            df_label2.rename(columns={old_col: new_col}, inplace=True)
    
    # 질문코드에 W2_ 접두사 추가 (Welcome 1과 구분)
    df_label2['질문코드'] = 'W2_' + df_label2['질문코드'].astype(str)
    
    print(f"✅ Welcome 2 해설지 처리 완료: {len(df_label2)}개 항목")
    
except Exception as e:
    print(f"❌ Welcome 2 해설지 처리 중 오류: {e}")
    df_label2 = pd.DataFrame()

# 두 Welcome 해설지 통합 및 저장
print("\n=== Welcome 데이터 사전 통합 ===")
if not df_label1.empty and not df_label2.empty:
    panel_data_dictionary = pd.concat([df_label1, df_label2], ignore_index=True)
    print(f"✅ Welcome 1 + Welcome 2 통합: {len(panel_data_dictionary)}개 항목")
elif not df_label1.empty:
    panel_data_dictionary = df_label1
    print(f"✅ Welcome 1만 사용: {len(panel_data_dictionary)}개 항목")
elif not df_label2.empty:
    panel_data_dictionary = df_label2
    print(f"✅ Welcome 2만 사용: {len(panel_data_dictionary)}개 항목")
else:
    panel_data_dictionary = pd.DataFrame()
    print("❌ 처리된 데이터가 없습니다.")

# Welcome 데이터 사전 저장
if not panel_data_dictionary.empty:
    dict_output_path = '../data/processed/panel_data_dictionary.csv'
    panel_data_dictionary.to_csv(dict_output_path, index=False, encoding='utf-8-sig')
    print(f"🎉 Welcome 데이터 사전 저장 완료: '{dict_output_path}'")
    print(f"📊 최종 데이터 사전 크기: {panel_data_dictionary.shape}")
    
    # 샘플 데이터 확인
    print("\n📋 Welcome 데이터 사전 샘플:")
    print(panel_data_dictionary.head())
else:
    print("❌ 저장할 Welcome 데이터 사전이 없습니다.")


=== Welcome 1 해설지 처리 ===
Welcome 1 해설지 원본 크기: (598, 3)
컬럼명: ['변수명', '문항', '문항유형']
✅ Welcome 1 해설지 처리 완료: 576개 항목

=== Welcome 2 해설지 처리 ===
Welcome 2 해설지 원본 크기: (7, 3)
컬럼명: ['변수명', '문항', '문항유형']
✅ Welcome 2 해설지 처리 완료: 7개 항목

=== Welcome 데이터 사전 통합 ===
✅ Welcome 1 + Welcome 2 통합: 583개 항목
🎉 Welcome 데이터 사전 저장 완료: '../data/processed/panel_data_dictionary.csv'
📊 최종 데이터 사전 크기: (583, 6)

📋 Welcome 데이터 사전 샘플:
    질문코드  질문내용  보기번호         보기내용   문항 문항유형
0  W1_Q1  결혼여부   1.0           미혼  NaN  NaN
1  W1_Q1  결혼여부   2.0           기혼  NaN  NaN
2  W1_Q1  결혼여부   3.0  기타(사별/이혼 등)  NaN  NaN
3  W1_Q3   가족수   1.0    1명(혼자 거주)  NaN  NaN
4  W1_Q3   가족수   2.0           2명  NaN  NaN


## 🔗 4. Welcome 파일 통합

Welcome 파일들을 먼저 통합하여 기본 DataFrame을 생성합니다.


In [42]:
# --- 1단계: 모든 파일에서 고유한 패널 ID(mb_sn) 전체 명단 만들기 ---
print("--- 1단계: 전체 패널 마스터 명단 생성 시작 ---")
all_panel_ids = set()

# 모든 파일에서 ID만 추출
all_files = glob.glob(os.path.join(base_path, '**/*.xlsx'), recursive=True)

for file in all_files:
    try:
        # 💡 header=1을 가정하고, '고유번호' 또는 'mb_sn' 컬럼만 읽어옵니다.
        # Welcome 파일에 대한 예외 처리를 추가합니다.
        header_row = 0 if 'Welcome' in file else 1
        df_id = pd.read_excel(file, header=header_row)
        
        if '고유번호' in df_id.columns:
            all_panel_ids.update(df_id['고유번호'].dropna().unique())
        elif 'mb_sn' in df_id.columns:
            all_panel_ids.update(df_id['mb_sn'].dropna().unique())
            
    except Exception as e:
        print(f"  [경고] {os.path.basename(file)} 파일 처리 중 오류: {e}")

# 고유한 ID 목록으로 최종 기준이 될 base_df 생성
base_df = pd.DataFrame(list(all_panel_ids), columns=['mb_sn'])
print(f"--- 마스터 명단 생성 완료! 총 고유 패널 수: {len(base_df)}명 ---")

# --- 2단계: 마스터 명단에 파일별 정보 채워넣기 (LEFT JOIN) ---
print("\n--- 2단계: 파일별 정보 병합 시작 (LEFT JOIN) ---")
welcome_files = glob.glob(os.path.join(base_path, 'Welcome/*.xlsx'))
quickpoll_files = glob.glob(os.path.join(base_path, 'Quickpoll/*.xlsx'))

# 먼저 Welcome 파일(핵심 인구통계 정보)을 LEFT JOIN으로 병합합니다.
for file in welcome_files:
    print(f"병합 중: {os.path.basename(file)}")
    temp_df = pd.read_excel(file) # Welcome은 header가 첫 줄에 있다고 가정
    
    # '고유번호' 컬럼을 'mb_sn'으로 변경
    if '고유번호' in temp_df.columns:
        temp_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    
    # Welcome 1, 2 컬럼명 중복 방지를 위한 접두사 처리
    if 'Welcome_1st' in file:
        # Welcome 1의 Q 컬럼에 W1_ 접두사 추가
        w1_columns = [col for col in temp_df.columns if col.startswith('Q')]
        for col in w1_columns:
            temp_df.rename(columns={col: 'W1_' + col}, inplace=True)
        print(f"  - Welcome 1 컬럼명 변경: {len(w1_columns)}개 컬럼")
    elif 'Welcome_2nd' in file:
        # Welcome 2의 Q 컬럼에 W2_ 접두사 추가
        w2_columns = [col for col in temp_df.columns if col.startswith('Q')]
        for col in w2_columns:
            temp_df.rename(columns={col: 'W2_' + col}, inplace=True)
        print(f"  - Welcome 2 컬럼명 변경: {len(w2_columns)}개 컬럼")
    
    # LEFT JOIN으로 안전하게 병합
    base_df = pd.merge(base_df, temp_df, on='mb_sn', how='left')
    print(f"  - 병합 후 크기: {base_df.shape}")
    
    # 메모리 정리
    del temp_df
    gc.collect()

# 다음으로 Quickpoll 파일을 LEFT JOIN으로 병합합니다.
common_info_cols = ['구분', 'mb_sn', '성별', '나이', '지역', '설문일시']
for file in quickpoll_files:
    filename = os.path.basename(file).split('.')[0]
    print(f"병합 중: {filename}.xlsx")
    temp_df = pd.read_excel(file, header=1)
    
    if '고유번호' in temp_df.columns:
        temp_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    
    if 'mb_sn' in temp_df.columns:
        # 설문 답변 컬럼만 추출 (주관식 답변도 포함)
        answer_cols = [col for col in temp_df.columns if col not in common_info_cols]
        
        # 주관식 답변 컬럼 식별 및 처리
        subjective_cols = [col for col in answer_cols if 'ETC' in col or '기타' in col or '직접입력' in col]
        objective_cols = [col for col in answer_cols if col not in subjective_cols]
        
        print(f"  - 객관식 답변: {len(objective_cols)}개")
        print(f"  - 주관식 답변: {len(subjective_cols)}개")
        
        # 컬럼명 변경
        rename_dict = {}
        for col in objective_cols:
            rename_dict[col] = f"{col}_{filename}"
        for col in subjective_cols:
            rename_dict[col] = f"{col}_TEXT_{filename}"  # 주관식은 _TEXT_ 접미사 추가
        
        temp_df.rename(columns=rename_dict, inplace=True)
        unique_answer_cols = list(rename_dict.values())
        merge_target_df = temp_df[['mb_sn'] + unique_answer_cols]
        
        # LEFT JOIN으로 안전하게 병합
        base_df = pd.merge(base_df, merge_target_df, on='mb_sn', how='left')
        print(f"  - 병합 후 크기: {base_df.shape}")
    
    # 메모리 정리
    del temp_df
    gc.collect()

print("🎉 최종 데이터 통합 완료!")
print(f"최종 데이터 크기: {base_df.shape[0]}개의 행, {base_df.shape[1]}개의 열")


--- 1단계: 전체 패널 마스터 명단 생성 시작 ---
--- 마스터 명단 생성 완료! 총 고유 패널 수: 36113명 ---

--- 2단계: 파일별 정보 병합 시작 (LEFT JOIN) ---
병합 중: Welcome_2nd.xlsx
  - Welcome 2 컬럼명 변경: 21개 컬럼
  - 병합 후 크기: (36113, 22)
병합 중: Welcome_1st.xlsx
  - Welcome 1 컬럼명 변경: 4개 컬럼
  - 병합 후 크기: (36172, 26)
병합 중: qpoll_join_250106.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 27)
병합 중: qpoll_join_250604.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 28)
병합 중: qpoll_join_250716.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 29)
병합 중: qpoll_join_250624.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 30)
병합 중: qpoll_join_250326.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 31)
병합 중: qpoll_join_250310.xlsx
  - 객관식 답변: 3개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 34)
병합 중: qpoll_join_250605.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 35)
병합 중: qpoll_join_250107.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병합 후 크기: (36172, 36)
병합 중: qpoll_join_250116.xlsx
  - 객관식 답변: 1개
  - 주관식 답변: 0개
  - 병

## 📊 5. Quickpoll 파일 통합

Quickpoll 파일들을 기본 DataFrame에 통합합니다.


In [ ]:
# Quickpoll 파일 통합 (2단계 접근법)
print("=== Quickpoll 파일 통합 시작 ===")
quickpoll_files = glob.glob(os.path.join(base_path, 'Quickpoll/*.xlsx'))
print(f"처리할 Quickpoll 파일 수: {len(quickpoll_files)}")

# 1단계: 모든 Quickpoll 파일에서 고유한 패널 ID 추출
print("\n--- 1단계: Quickpoll 패널 ID 추출 ---")
quickpoll_panel_ids = set()

for file in quickpoll_files:
    try:
        temp_df = pd.read_excel(file, header=1)
        if '고유번호' in temp_df.columns:
            quickpoll_panel_ids.update(temp_df['고유번호'].dropna().unique())
        elif 'mb_sn' in temp_df.columns:
            quickpoll_panel_ids.update(temp_df['mb_sn'].dropna().unique())
    except Exception as e:
        print(f"  [경고] {os.path.basename(file)} 파일 처리 중 오류: {e}")

print(f"Quickpoll에서 발견된 고유 패널 수: {len(quickpoll_panel_ids)}")

# 2단계: Quickpoll 파일들을 하나씩 base_df에 LEFT JOIN
print("\n--- 2단계: Quickpoll 파일 통합 ---")
common_info_cols = ['구분', 'mb_sn', '성별', '나이', '지역', '설문일시']

for file in quickpoll_files:
    filename = os.path.basename(file).split('.')[0]
    print(f"병합 중: {filename}.xlsx")
    
    try:
        temp_df = pd.read_excel(file, header=1)
        
        if '고유번호' in temp_df.columns:
            temp_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
        
        if 'mb_sn' in temp_df.columns:
            # 설문 답변 컬럼만 추출 (주관식 답변도 포함)
            answer_cols = [col for col in temp_df.columns if col not in common_info_cols]
            
            # 주관식 답변 컬럼 식별 및 처리
            subjective_cols = [col for col in answer_cols if 'ETC' in col or '기타' in col or '직접입력' in col]
            objective_cols = [col for col in answer_cols if col not in subjective_cols]
            
            print(f"  - 객관식 답변: {len(objective_cols)}개")
            print(f"  - 주관식 답변: {len(subjective_cols)}개")
            
            # 컬럼명 변경
            rename_dict = {}
            for col in objective_cols:
                rename_dict[col] = f"{col}_{filename}"
            for col in subjective_cols:
                rename_dict[col] = f"{col}_TEXT_{filename}"  # 주관식은 _TEXT_ 접미사 추가
            
            temp_df.rename(columns=rename_dict, inplace=True)
            unique_answer_cols = list(rename_dict.values())
            merge_target_df = temp_df[['mb_sn'] + unique_answer_cols]
            
            # LEFT JOIN으로 안전하게 병합
            base_df = pd.merge(base_df, merge_target_df, on='mb_sn', how='left')
            print(f"  - 병합 후 크기: {base_df.shape}")
        
        # 메모리 정리
        del temp_df
        gc.collect()
        
    except Exception as e:
        print(f"  [오류] {filename} 처리 중 오류: {e}")

print("🎉 Quickpoll 파일 통합 완료!")
print(f"최종 데이터 크기: {base_df.shape[0]}개의 행, {base_df.shape[1]}개의 열")


=== Quickpoll 파일 통합 시작 ===
처리할 Quickpoll 파일 수: 35

--- 1단계: Quickpoll 패널 ID 추출 ---


## 📋 6. Quickpoll 해설지 추출

Quickpoll 파일들의 해설지를 처리하여 데이터 사전을 생성합니다.


In [ ]:
# Quickpoll 해설지 추출 및 데이터 사전 생성
print("=== Quickpoll 해설지 추출 ===")

quickpoll_dictionary = []

for file in quickpoll_files:
    filename = os.path.basename(file).split('.')[0]
    print(f"해설지 추출 중: {filename}.xlsx")
    
    try:
        # Quickpoll 파일의 두 번째 시트(해설지) 읽기
        df_label_raw = pd.read_excel(file, sheet_name=1)
        print(f"  - 해설지 원본 크기: {df_label_raw.shape}")
        print(f"  - 컬럼명: {df_label_raw.columns.tolist()}")
        
        # Quickpoll 해설지 형식 처리 (설문제목, 보기1, 보기2, ...)
        if '설문제목' in df_label_raw.columns:
            # 각 행(질문)에 대해 처리
            for idx, row in df_label_raw.iterrows():
                if pd.notna(row['설문제목']):
                    question_title = str(row['설문제목']).strip()
                    question_code = f"문항1_{filename}"  # 기본 질문 코드
                    
                    # 보기 컬럼들 찾기 (보기1, 보기2, ...)
                    option_cols = [col for col in df_label_raw.columns if col.startswith('보기') and not col.endswith('_CNT')]
                    
                    # 질문 추가
                    quickpoll_dictionary.append({
                        '질문코드': question_code,
                        '질문내용': question_title,
                        '보기번호': '',
                        '보기내용': '',
                        '파일명': filename
                    })
                    
                    # 각 보기 처리
                    for i, col in enumerate(option_cols, 1):
                        if pd.notna(row[col]) and str(row[col]).strip():
                            option_text = str(row[col]).strip()
                            quickpoll_dictionary.append({
                                '질문코드': question_code,
                                '질문내용': question_title,
                                '보기번호': i,
                                '보기내용': option_text,
                                '파일명': filename
                            })
            
            print(f"  - 추출된 항목: {len([d for d in quickpoll_dictionary if d['파일명'] == filename])}개")
        else:
            print(f"  - [경고] {filename}에서 '설문제목' 컬럼을 찾을 수 없습니다.")
            
    except Exception as e:
        print(f"  - [오류] {filename} 해설지 처리 중 오류: {e}")

# Quickpoll 데이터 사전 생성 및 저장
if quickpoll_dictionary:
    quickpoll_df = pd.DataFrame(quickpoll_dictionary)
    
    # Quickpoll 데이터 사전 저장
    quickpoll_dict_path = '../data/processed/quickpoll_data_dictionary.csv'
    quickpoll_df.to_csv(quickpoll_dict_path, index=False, encoding='utf-8-sig')
    print(f"\n🎉 Quickpoll 데이터 사전 저장 완료: '{quickpoll_dict_path}'")
    print(f"📊 총 {len(quickpoll_df)}개 항목 추출")
    
    # Welcome과 Quickpoll 데이터 사전 통합
    if 'panel_data_dictionary' in locals() and not panel_data_dictionary.empty:
        # Welcome 데이터 사전에 파일명 컬럼 추가
        panel_data_dictionary['파일명'] = 'Welcome'
        
        # 두 데이터 사전 통합
        combined_dictionary = pd.concat([panel_data_dictionary, quickpoll_df], ignore_index=True)
        
        # 통합 데이터 사전 저장
        combined_dict_path = '../data/processed/00_master_mapping_table.csv'
        combined_dictionary.to_csv(combined_dict_path, index=False, encoding='utf-8-sig')
        print(f"🎉 통합 데이터 사전 저장 완료: '{combined_dict_path}'")
        print(f"📊 최종 통합 데이터 사전 크기: {combined_dictionary.shape}")
        
        # 샘플 데이터 확인
        print(f"\n📋 통합 데이터 사전 샘플:")
        print(combined_dictionary.head())
    else:
        print("❌ Welcome 데이터 사전이 없어 통합할 수 없습니다.")
else:
    print("❌ 추출된 Quickpoll 해설지 데이터가 없습니다.")


=== Quickpoll 해설지 추출 ===
해설지 추출 중: qpoll_join_250106.xlsx
  - 해설지 원본 크기: (1, 22)
  - 컬럼명: ['설문제목', '보기1', '보기2', '보기3', '보기4', '보기5', '보기6', '보기7', '보기8', '보기9', '보기10', '총참여자수', '보기1_CNT', '보기2_CNT', '보기3_CNT', '보기4_CNT', '보기5_CNT', '보기6_CNT', '보기7_CNT', '보기8_CNT', '보기9_CNT', '보기10_CNT']
  - 추출된 항목: 11개
해설지 추출 중: qpoll_join_250604.xlsx
  - 해설지 원본 크기: (1, 16)
  - 컬럼명: ['설문제목', '보기1', '보기2', '보기3', '보기4', '보기5', '보기6', '보기7', '총참여자수', '보기1_CNT', '보기2_CNT', '보기3_CNT', '보기4_CNT', '보기5_CNT', '보기6_CNT', '보기7_CNT']
  - 추출된 항목: 8개
해설지 추출 중: qpoll_join_250716.xlsx
  - 해설지 원본 크기: (1, 20)
  - 컬럼명: ['설문제목', '보기1', '보기2', '보기3', '보기4', '보기5', '보기6', '보기7', '보기8', '보기9', '총참여자수', '보기1_CNT', '보기2_CNT', '보기3_CNT', '보기4_CNT', '보기5_CNT', '보기6_CNT', '보기7_CNT', '보기8_CNT', '보기9_CNT']
  - 추출된 항목: 10개
해설지 추출 중: qpoll_join_250624.xlsx
  - 해설지 원본 크기: (1, 12)
  - 컬럼명: ['설문제목', '보기1', '보기2', '보기3', '보기4', '보기5', '총참여자수', '보기1_CNT', '보기2_CNT', '보기3_CNT', '보기4_CNT', '보기5_CNT']
  - 추출된 항목: 6개
해설지 추출 중: qpoll_join_

## 🔍 7. 중복 데이터 처리

데이터 품질을 개선하기 위해 중복된 mb_sn을 확인하고 제거합니다.


In [ ]:
# 데이터 중복 확인 및 분석
print("=== 데이터 중복 분석 ===")
print(f"전체 행 수: {len(base_df)}")
print(f"고유한 mb_sn 개수: {base_df['mb_sn'].nunique()}")
print(f"중복된 mb_sn 개수: {len(base_df) - base_df['mb_sn'].nunique()}")

# 중복된 mb_sn 확인
duplicated_sn = base_df[base_df.duplicated(subset=['mb_sn'], keep=False)]
print(f"\n중복된 mb_sn이 있는 행 수: {len(duplicated_sn)}")

if len(duplicated_sn) > 0:
    print("\n중복된 mb_sn 예시:")
    print(duplicated_sn[['mb_sn']].head(10))
    
    # 중복 제거 (첫 번째 값만 유지)
    print("\n중복 제거 중...")
    base_df_clean = base_df.drop_duplicates(subset=['mb_sn'], keep='first')
    print(f"중복 제거 후 행 수: {len(base_df_clean)}")
    
    # 정리된 데이터 저장
    output_path_clean = '../data/processed/01_consolidated_data.csv'
    base_df_clean.to_csv(output_path_clean, index=False, encoding='utf-8-sig')
    print(f"중복 제거된 데이터가 '{output_path_clean}'에 저장되었습니다.")
    
    # 원본 데이터도 업데이트
    base_df = base_df_clean
    print(f"base_df가 중복 제거된 데이터로 업데이트되었습니다.")
else:
    print("중복된 데이터가 없습니다.")


=== 데이터 중복 분석 ===
전체 행 수: 36113
고유한 mb_sn 개수: 36113
중복된 mb_sn 개수: 0

중복된 mb_sn이 있는 행 수: 0
중복된 데이터가 없습니다.


## 💾 8. 최종 결과 저장

통합된 데이터를 최종적으로 저장하고 결과를 확인합니다.


In [ ]:
# --- 3단계: 최종 결과 확인 및 저장 ---
print("\n--- 3단계: 최종 결과 확인 ---")
print(f"최종 데이터 크기: {base_df.shape[0]}개의 행, {base_df.shape[1]}개의 열")

# 데이터 정보 확인
print("\n--- 데이터 정보 ---")
base_df.info()

# 데이터 미리보기
print("\n--- 데이터 미리보기 ---")
display(base_df.head())

# 메모리 사용량 확인
final_memory = base_df.memory_usage(deep=True).sum() / 1024**2
print(f"\n최종 메모리 사용량: {final_memory:.1f}MB")

# 최종 결과물 저장
output_dir = '../data/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"폴더 생성: {output_dir}")

output_path = '../data/processed/01_consolidated_data.csv'
base_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"통합된 데이터가 '{output_path}' 경로에 저장되었습니다.")
print(f"저장된 파일 크기: {base_df.shape}")

print("\n🎉 데이터 통합 및 전처리 작업 완료!")
print("=" * 50)



--- 3단계: 최종 결과 확인 ---
최종 데이터 크기: 36113개의 행, 42개의 열

--- 데이터 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36113 entries, 0 to 36112
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mb_sn                  36113 non-null  object 
 1   문항1_qpoll_join_250106  5057 non-null   object 
 2   문항1_qpoll_join_250604  4954 non-null   float64
 3   문항1_qpoll_join_250716  4561 non-null   float64
 4   문항1_qpoll_join_250624  4983 non-null   float64
 5   문항1_qpoll_join_250326  5005 non-null   object 
 6   문항1_qpoll_join_250310  5028 non-null   object 
 7   문항2_qpoll_join_250310  4983 non-null   float64
 8   문항3_qpoll_join_250310  4969 non-null   float64
 9   문항1_qpoll_join_250605  5026 non-null   float64
 10  문항1_qpoll_join_250107  5048 non-null   float64
 11  문항1_qpoll_join_250116  5118 non-null   float64
 12  문항1_qpoll_join_250317  5013 non-null   object 
 13  문항2_qpoll_join_250317  4983 non-null  

,mb_sn,문항1_qpoll_join_250106,문항1_qpoll_join_250604,문항1_qpoll_join_250716,문항1_qpoll_join_250624,문항1_qpoll_join_250326,문항1_qpoll_join_250310,문항2_qpoll_join_250310,문항3_qpoll_join_250310,문항1_qpoll_join_250605,...,문항1_qpoll_join_250714,문항1_qpoll_join_250221,문항1_qpoll_join_250626,문항1_qpoll_join_250723,문항1_qpoll_join_250627,문항1_qpoll_join_250328,문항1_qpoll_join_250703,문항1_qpoll_join_250611,문항1_qpoll_join_250304,문항2_qpoll_join_250304
0,w236284973740997,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,w292274837502403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,w1095166895549,10,6.0,7.0,1.0,3,4,4.0,6.0,3.0,...,1.0,4.0,5,3.0,2.0,6.0,4,3.0,4.0,3.0
3,w166982084763719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,w179334670473409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



최종 메모리 사용량: 23.3MB
통합된 데이터가 '../data/processed/01_consolidated_data.csv' 경로에 저장되었습니다.
저장된 파일 크기: (36113, 42)

🎉 데이터 통합 및 전처리 작업 완료!
